## 1. Carga de librerías y configuración inicial

En este notebook voy a explorar, limpiar y unificar los datasets industriales de la provincia de Tierra del Fuego.  
El objetivo es dejar todo preparado para construir un modelo de regresión supervisada que me permita predecir la producción mensual por sector, a partir de la cantidad de empleados, establecimientos y variables temporales como el mes y el año.

In [1]:
# Importo las librerías que voy a usar para el análisis y procesamiento de datos
import pandas as pd
import numpy as np

# Defino la ruta base donde tengo los archivos crudos
ruta_base = '../data/raw/'


## 2. Dataset de Producción por Sector Industrial

En esta sección cargo y trabajo con el dataset de producción mensual, desagregado por sector.  
Voy a realizar una exploración inicial para entender su estructura, luego una limpieza de columnas y valores, y por último la normalización.


### 2.1 Exploración inicial del dataset de Producción

El archivo Excel de producción contiene varias hojas, una por sector industrial.  
Cada hoja tiene una estructura ligeramente distinta, así que primero defino cuántas filas debo saltear para acceder correctamente a los datos.  
Luego recorro las hojas, selecciono las columnas relevantes, las normalizo y las guardo en una lista para combinarlas al final.



In [2]:
# Defino la ruta del archivo Excel
archivo = ruta_base + '14_3_03_Produccion_Industrial-1-1.xlsx'

# Diccionario con nombre de hoja y cantidad de filas a saltear (skiprows)
hojas_y_skips = {
    'CONFECCIONISTA': 5,
    'ELECTRONICA': 5,
    'PESQUERA': 5,
    'TEXTIL': 5,
    'PLASTICA': 5,
    'OTROS': 4
}

### 2.2 Preparación inicial: renombrado, tipos y unificación

Una vez cargados los datos de cada hoja, realizo una serie de transformaciones básicas:  
renombro las columnas, creo una columna nueva con el nombre del sector, convierto los tipos de datos  
y finalmente uno todos los DataFrames en una sola tabla general llamada `df_produccion_total`.  
Reordeno las columnas para dejar el formato estandarizado.


In [3]:
# Lista donde voy a guardar los DataFrames de cada hoja
lista_df = []

for hoja, skip in hojas_y_skips.items():
    print(f"\nLeyendo hoja: {hoja} (skiprows={skip})")
    df_temp = pd.read_excel(archivo, sheet_name=hoja, skiprows=skip)

    try:
        # Las hojas tienen la columna de Producción en distinta ubicación
        if hoja == 'OTROS':
            df_temp = df_temp.iloc[:, [0, 1, 4]]  # Año, Mes, Producción (en columna E)
        else:
            df_temp = df_temp.iloc[:, [0, 1, 2]]  # Año, Mes, Producción (columna C)
        
        # Renombro columnas
        df_temp.columns = ['anio', 'mes', 'Produccion']
        
        # Relleno valores faltantes en el año
        df_temp['anio'] = df_temp['anio'].fillna(method='ffill')
        
        
        # Elimino filas que no tienen datos relevantes
        df_temp = df_temp.dropna(subset=['anio', 'mes', 'Produccion'])
        
        # Convierto tipos
        df_temp['anio'] = df_temp['anio'].astype(int)
        df_temp['mes'] = df_temp['mes'].astype(str)
        
        # Agrego la columna de sector con el nombre de la hoja (capitalizado)
        df_temp['sector'] = hoja.capitalize()
        
        # Agrego el DataFrame a la lista
        lista_df.append(df_temp)
        
    except Exception as e:
        print(f"Error en hoja {hoja}: {e}")


Leyendo hoja: CONFECCIONISTA (skiprows=5)


C:\Users\Luciano\AppData\Local\Temp\ipykernel_2704\3278948592.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')



Leyendo hoja: ELECTRONICA (skiprows=5)


C:\Users\Luciano\AppData\Local\Temp\ipykernel_2704\3278948592.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')
C:\Users\Luciano\AppData\Local\Temp\ipykernel_2704\3278948592.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')



Leyendo hoja: PESQUERA (skiprows=5)

Leyendo hoja: TEXTIL (skiprows=5)


C:\Users\Luciano\AppData\Local\Temp\ipykernel_2704\3278948592.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')



Leyendo hoja: PLASTICA (skiprows=5)

Leyendo hoja: OTROS (skiprows=4)


C:\Users\Luciano\AppData\Local\Temp\ipykernel_2704\3278948592.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')
C:\Users\Luciano\AppData\Local\Temp\ipykernel_2704\3278948592.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')


Una vez procesadas todas las hojas, uno los DataFrames en uno solo, reorganizo las columnas y muestro una vista previa.


In [4]:
# Uno todos los sectores en un único DataFrame
df_produccion_total = pd.concat(lista_df, ignore_index=True)

# Reordeno columnas
df_produccion_total = df_produccion_total[['anio', 'mes', 'sector', 'Produccion']]

# Vista previa del resultado final
print("✅ Producción final normalizada y completa:")
display(df_produccion_total.head(15))

✅ Producción final normalizada y completa:


,anio,mes,sector,Produccion
0,2013,Ene,Confeccionista,219953.0
1,2013,Feb,Confeccionista,478428.0
2,2013,Mar,Confeccionista,455248.0
3,2013,Apr,Confeccionista,569380.0
4,2013,May,Confeccionista,610624.0
5,2013,Jun,Confeccionista,517452.0
6,2013,Jul,Confeccionista,559855.0
7,2013,Ago,Confeccionista,579140.0
8,2013,Sep,Confeccionista,481588.0
9,2013,Oct,Confeccionista,517940.0


### 2.3 Diagnóstico del dataset unificado de Producción

Antes de aplicar cualquier transformación adicional, realizo un diagnóstico del DataFrame `df_produccion_total`  
para verificar tipos de datos, presencia de valores nulos, duplicados y distribución de los valores.  
Esto me permite asegurar que todo esté en orden antes de proceder con la limpieza final.


In [5]:
# Tipo de dato y cantidad de valores no nulos
df_produccion_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   anio        881 non-null    int32 
 1   mes         881 non-null    object
 2   sector      881 non-null    object
 3   Produccion  881 non-null    object
dtypes: int32(1), object(3)
memory usage: 24.2+ KB


In [6]:
# Cantidad de valores nulos por columna
df_produccion_total.isnull().sum()

anio          0
mes           0
sector        0
Produccion    0
dtype: int64

In [7]:
# Cantidad de filas duplicadas
df_produccion_total.duplicated().sum()

0

In [8]:
# Verifico la cantidad de sectores distintos y sus nombres
print("Sectores disponibles:")
print(df_produccion_total['sector'].value_counts())

Sectores disponibles:
sector
Confeccionista    147
Electronica       147
Pesquera          147
Textil            147
Otros             147
Plastica          146
Name: count, dtype: int64


In [9]:
# Verifico los valores únicos de mes y su frecuencia
print("Distribución de registros por mes:")
print(df_produccion_total['mes'].value_counts())


Distribución de registros por mes:
mes
Sep     60
Oct     60
Nov     60
Dic     60
May     55
Ene     54
Mar     54
Apr     54
Jun     54
Jul     54
Ago     54
Feb     54
Jun     12
Feb*    12
Ene*    12
Ago     12
Jul     12
May     12
Feb     12
Mar     12
Apr     12
Ene     12
Mar*    11
Dic      6
Nov      6
Oct      6
Sep      6
Abr*     6
Jun*     6
Jul*     6
Ago*     6
Sep*     6
Oct*     6
Nov*     6
Dic*     6
May*     5
Name: count, dtype: int64


### 2.4 Limpieza final

Antes de finalizar esta etapa, realicé dos ajustes clave sobre el dataset `df_produccion_total`:

- Corregí los valores de la columna `mes`, ya que algunos contenían símbolos extraños como `"Ene*"` o `"May*"`. Ahora todos los meses están normalizados en formato texto (`'Ene'`, `'Feb'`, etc.).
- Traté los valores no válidos de la columna `Produccion`, como `'-'`, transformándolos en `NaN`. En lugar de eliminar esos registros, decidí conservarlos completando el valor faltante con el anterior dentro del mismo sector (`ffill`), ya que esos registros son valiosos para el análisis.

Una vez que todos los valores de producción estuvieron completos, los redondeé y los convertí a tipo entero, para asegurar consistencia en el análisis posterior.


In [10]:
# Diccionario base para normalizar abreviaciones o errores
meses_validos = {
    'ene': 'enero', 'feb': 'febrero', 'mar': 'marzo',
    'abr': 'abril', 'may': 'mayo', 'jun': 'junio',
    'jul': 'julio', 'ago': 'agosto', 'sep': 'septiembre',
    'oct': 'octubre', 'nov': 'noviembre', 'dic': 'diciembre', 'apr':'abril'
}

# Limpieza unificada
df_produccion_total['mes'] = (
    df_produccion_total['mes']
    .astype(str)
    .str.lower()
    .str.strip()
    .str.replace(r'\*', '', regex=True)
    .str.replace(r'\s+', '', regex=True)
    .map(lambda x: next((v for k, v in meses_validos.items() if x.startswith(k)), x))
)


In [11]:
# Verificar los valores únicos de la columna 'mes' en producción
print(sorted(df_produccion_total['mes'].unique()))

['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']


In [12]:
# Convierto valores no numéricos a NaN
df_produccion_total['Produccion'] = pd.to_numeric(df_produccion_total['Produccion'], errors='coerce')

# Ordeno para que el fillna sea correcto por sector y fecha
df_produccion_total = df_produccion_total.sort_values(by=['sector', 'anio', 'mes'])

# Completo valores nulos con el anterior disponible dentro del mismo sector
df_produccion_total['Produccion'] = df_produccion_total.groupby('sector')['Produccion'].fillna(method='ffill')

# Redondeo y convierto a entero
df_produccion_total['Produccion'] = df_produccion_total['Produccion'].round().astype(int)


C:\Users\Luciano\AppData\Local\Temp\ipykernel_2704\1023895956.py:8: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df_produccion_total['Produccion'] = df_produccion_total.groupby('sector')['Produccion'].fillna(method='ffill')
C:\Users\Luciano\AppData\Local\Temp\ipykernel_2704\1023895956.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_produccion_total['Produccion'] = df_produccion_total.groupby('sector')['Produccion'].fillna(method='ffill')


In [13]:
# Verifico el tipo de dato y la cantidadd de nulos
df_produccion_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 881 entries, 3 to 587
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   anio        881 non-null    int32 
 1   mes         881 non-null    object
 2   sector      881 non-null    object
 3   Produccion  881 non-null    int32 
dtypes: int32(2), object(2)
memory usage: 27.5+ KB


Ordeno el DataFrame cronológicamente usando la columna 'mes' como categoría ordenada temporalmente.

Esto asegura que los meses se ordenen en su secuencia real (enero → diciembre) y no alfabéticamente.

Luego, vuelvo a convertir la columna 'mes' a tipo texto (object) para mantener la consistencia con el resto del análisis.

In [14]:
# Defino el orden cronológico de los meses
orden_meses = [
    'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
    'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre'
]

# Convierto temporalmente a categoría ordenada para ordenar correctamente
df_produccion_total['mes'] = pd.Categorical(df_produccion_total['mes'], categories=orden_meses, ordered=True)

# Ordeno cronológicamente por sector, año y mes
df_produccion_total = df_produccion_total.sort_values(by=['sector', 'anio', 'mes']).reset_index(drop=True)

# Vuelvo a convertir la columna 'mes' a tipo object (texto plano)
df_produccion_total['mes'] = df_produccion_total['mes'].astype(str)


In [15]:
# Verifico el tipo de dato y la cantidadd de nulos
df_produccion_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   anio        881 non-null    int32 
 1   mes         881 non-null    object
 2   sector      881 non-null    object
 3   Produccion  881 non-null    int32 
dtypes: int32(2), object(2)
memory usage: 20.8+ KB


In [16]:
# Muestro los primeros registros de cada sector para verificar desde qué año arranca cada uno
for sector in df_produccion_total['sector'].unique():
    print(f"\nPrimeras filas del sector: {sector}")
    display(df_produccion_total[df_produccion_total['sector'] == sector].head())



Primeras filas del sector: Confeccionista


,anio,mes,sector,Produccion
0,2013,enero,Confeccionista,219953
1,2013,febrero,Confeccionista,478428
2,2013,marzo,Confeccionista,455248
3,2013,abril,Confeccionista,569380
4,2013,mayo,Confeccionista,610624



Primeras filas del sector: Electronica


,anio,mes,sector,Produccion
147,2013,enero,Electronica,1260521
148,2013,febrero,Electronica,1559546
149,2013,marzo,Electronica,1935759
150,2013,abril,Electronica,2072806
151,2013,mayo,Electronica,2538083



Primeras filas del sector: Otros


...

,anio,mes,sector,Produccion
441,2013,enero,Pesquera,2056519
442,2013,febrero,Pesquera,2016742
443,2013,marzo,Pesquera,1658171
444,2013,abril,Pesquera,2612904
445,2013,mayo,Pesquera,1993512



Primeras filas del sector: Plastica


,anio,mes,sector,Produccion
588,2013,enero,Plastica,7415631
589,2013,febrero,Plastica,7822623
590,2013,marzo,Plastica,8822137
591,2013,abril,Plastica,9082189
592,2013,mayo,Plastica,8274301



Primeras filas del sector: Textil


,anio,mes,sector,Produccion
734,2013,enero,Textil,2067922
735,2013,febrero,Textil,2210462
736,2013,marzo,Textil,2999587
737,2013,abril,Textil,2624834
738,2013,mayo,Textil,2943540


In [17]:
# Verifico que luego de la limpieza mantengo la misma cantidad de registros sin haber perdido nada
df_produccion_total['sector'].value_counts()

sector
Confeccionista    147
Electronica       147
Otros             147
Pesquera          147
Textil            147
Plastica          146
Name: count, dtype: int64

## 3. Dataset de Empleados por Sector Industrial

En esta sección trabajo con el archivo que contiene la cantidad de personal ocupado en establecimientos industriales, desagregado por sector.  
El objetivo es dejarlo en formato largo, con columnas normalizadas y sin valores faltantes, listo para ser combinado con el resto de los datasets.


### 3.1 Carga del archivo y selección de columnas

Cargo el archivo Excel original y recorto solo las columnas necesarias.  
Relleno los años faltantes hacia abajo y elimino filas vacías o incompletas.


In [18]:
# Cargo el archivo
archivo_empleados = ruta_base + '14_3_01_Personal_industria_rama-1.xlsx'
df_emp = pd.read_excel(archivo_empleados, skiprows=2)

# Corto solo las columnas necesarias (A → H = 0:8)
df_emp = df_emp.iloc[:, 0:8]
df_emp.columns = ['anio', 'mes', 'Confeccionista', 'Electrónicas', 'Plásticas', 'Textiles', 'Pesqueras', 'Otras']

# Relleno año hacia abajo
df_emp['anio'] = df_emp['anio'].ffill()

# Quito filas vacías de año o mes
df_emp = df_emp.dropna(subset=['anio', 'mes'])
df_emp['anio'] = df_emp['anio'].astype(int)
df_emp['mes'] = df_emp['mes'].astype(str)

### 3.2 Transformación a formato largo

Transformo el dataset a formato largo (una fila por mes, año y sector).  
Renombro las variables y capitalizo los nombres de sector.


In [19]:
# Paso a formato largo
df_empleados_limpio = df_emp.melt(
    id_vars=['anio', 'mes'],
    var_name='sector',
    value_name='empleados'
)

# Capitalizo el sector
df_empleados_limpio['sector'] = df_empleados_limpio['sector'].str.capitalize()

# Muestro el dataframe 
print("Dataset de empleados normalizado:")
display(df_empleados_limpio.head(15))


Dataset de empleados normalizado:


,anio,mes,sector,empleados
0,2001,Enero,Confeccionista,326.0
1,2001,Febrero,Confeccionista,323.0
2,2001,Marzo,Confeccionista,333.0
3,2001,Abril,Confeccionista,337.0
4,2001,Mayo,Confeccionista,346.0
5,2001,Junio,Confeccionista,352.0
6,2001,Julio,Confeccionista,354.0
7,2001,Agosto,Confeccionista,351.0
8,2001,Septiembre,Confeccionista,347.0
9,2001,Octubre,Confeccionista,339.0


### 3.3 Diagnóstico del dataset en formato largo

Una vez transformado el dataset al formato largo, realizo un diagnóstico inicial para entender su estructura.  
Reviso los tipos de datos, la presencia de valores nulos, duplicados, y la distribución general de sectores, meses y empleados.  
Esto me ayuda a detectar errores de carga o inconsistencias antes de comenzar la limpieza.


In [20]:
# Tipo de datos y cantidad de valores no nulos
df_empleados_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   anio       1746 non-null   int32 
 1   mes        1746 non-null   object
 2   sector     1746 non-null   object
 3   empleados  1746 non-null   object
dtypes: int32(1), object(3)
memory usage: 47.9+ KB


In [21]:
# Cantidad de valores nulos por columna
df_empleados_limpio.isnull().sum()

anio         0
mes          0
sector       0
empleados    0
dtype: int64

In [22]:
# Estadísticas descriptivas de los empleados
df_empleados_limpio['empleados'].value_counts()

empleados
296.0      8
723.0      8
722.0      7
192.0      7
126.0      7
          ..
10202.0    1
10152.0    1
9713.0     1
9335.0     1
551.0      1
Name: count, Length: 875, dtype: int64

In [23]:
# Conteo de registros por sector
df_empleados_limpio['sector'].value_counts()

sector
Confeccionista    291
Electrónicas      291
Plásticas         291
Textiles          291
Pesqueras         291
Otras             291
Name: count, dtype: int64

In [24]:
# Distribución de valores por mes
df_empleados_limpio['mes'].value_counts()

mes
julio          120
noviembre      120
junio          120
mayo           120
abril          120
marzo          120
febrero        120
enero          120
octubre        120
diciembre      120
agosto         120
septiembre     120
febrero         12
marzo           12
abril           12
mayo            12
junio           12
julio           12
agosto          12
enero           12
Enero*          12
Diciembre       12
Noviembre       12
Octubre         12
Septiembre      12
Febrero*        12
Junio*           6
Julio*           6
Agosto*          6
Septiembre*      6
Octubre*         6
Noviembre*       6
diciembre*       6
Mayo*            6
Abril*           6
Marzo*           6
Enero            6
diciembre        6
noviembre        6
octubre          6
septiembre       6
Febrero          6
Agosto           6
Julio            6
Junio            6
Mayo             6
Abril            6
Marzo            6
Mar*             6
Name: count, dtype: int64

In [25]:
# Verifico años disponibles
df_empleados_limpio['anio'].value_counts().sort_index()

anio
2001    72
2002    72
2003    72
2004    72
2005    72
2006    72
2007    72
2008    72
2009    72
2010    72
2011    72
2012    72
2013    72
2014    72
2015    72
2016    72
2017    72
2018    72
2019    72
2020    72
2021    72
2022    72
2023    72
2024    72
2025    18
Name: count, dtype: int64

In [26]:
# Duplicados
df_empleados_limpio.duplicated().sum()

0

### 3.3 Limpieza de la columna `empleados`

Convierto todos los valores no numéricos a `NaN`, y verifico cuántos valores nulos hay por sector.


In [27]:
df_empleados_limpio['empleados'] = pd.to_numeric(df_empleados_limpio['empleados'], errors='coerce')
df_empleados_limpio.isna().groupby(df_empleados_limpio['sector']).sum()

,anio,mes,sector,empleados
sector,,,,
Confeccionista,0,0,0,0
Electrónicas,0,0,0,0
Otras,0,0,0,0
Pesqueras,0,0,0,3
Plásticas,0,0,0,0
Textiles,0,0,0,0


### 3.4 Normalización de la columna `mes`

Unifico todos los valores de la columna `mes` a su forma completa y en minúscula.  
Esto evita inconsistencias como `"Feb"`, `"febrero"`, `"Feb*"` o `"feb.`".

In [28]:
mes_completo = {
    'ene': 'enero', 'enero': 'enero',
    'feb': 'febrero', 'febrero': 'febrero',
    'mar': 'marzo', 'marzo': 'marzo',
    'abr': 'abril', 'abril': 'abril',
    'may': 'mayo', 'mayo': 'mayo',
    'jun': 'junio', 'junio': 'junio',
    'jul': 'julio', 'julio': 'julio',
    'ago': 'agosto', 'agosto': 'agosto',
    'sep': 'septiembre', 'septiembre': 'septiembre',
    'oct': 'octubre', 'octubre': 'octubre',
    'nov': 'noviembre', 'noviembre': 'noviembre',
    'dic': 'diciembre', 'diciembre': 'diciembre'
}

df_empleados_limpio['mes'] = (
    df_empleados_limpio['mes']
    .astype(str)
    .str.strip()
    .str.lower()
    .str.replace(r'\*', '', regex=True)
    .map(mes_completo)
)


### 3.5 Imputación manual de valores faltantes

Para el sector Pesqueras, en los meses de diciembre de los años 2001, 2002 y 2004, imputo los valores faltantes.  
Utilizo el promedio entre el mes anterior y el posterior para mantener la coherencia temporal.




Diciembre 2001 → promedio entre noviembre 2001 y enero 2002

Diciembre 2002 → promedio entre noviembre 2002 y enero 2003

Diciembre 2004 → promedio entre noviembre 2004 y enero 2005



In [29]:
# Lista de casos a imputar manualmente
casos = [
    {'anio': 2001, 'mes': 'diciembre'},
    {'anio': 2002, 'mes': 'diciembre'},
    {'anio': 2004, 'mes': 'diciembre'}
]

for caso in casos:
    anio = caso['anio']
    mes = caso['mes']

    # Índice de la fila a reemplazar
    idx = df_empleados_limpio[
        (df_empleados_limpio['anio'] == anio) &
        (df_empleados_limpio['mes'] == mes) &
        (df_empleados_limpio['sector'] == 'Pesqueras')
    ].index

    if not idx.empty:
        i = idx[0]

        # Valor anterior y posterior
        empleados_ant = df_empleados_limpio.iloc[i - 1]['empleados']
        empleados_sig = df_empleados_limpio.iloc[i + 1]['empleados']

        # Promedio
        promedio = (empleados_ant + empleados_sig) / 2

        # Asignamos
        df_empleados_limpio.at[i, 'empleados'] = promedio

In [30]:
# Redondeo y convierto a entero para dejar la columna 'empleados' completamente limpia
# Esto es importante para que no queden valores flotantes luego de la imputación por promedio
df_empleados_limpio['empleados'] = df_empleados_limpio['empleados'].round().astype(int)

### 3.6 Verificación final del dataset

Verifico que no queden valores nulos, reviso los primeros registros por sector y confirmo que la columna `empleados` esté completamente limpia.


In [31]:
df_empleados_limpio.isna().groupby(df_empleados_limpio['sector']).sum()
df_empleados_limpio['sector'].value_counts()

sector
Confeccionista    291
Electrónicas      291
Plásticas         291
Textiles          291
Pesqueras         291
Otras             291
Name: count, dtype: int64

In [32]:
# Cuántos valores nulos hay por sector
df_empleados_limpio.isna().groupby(df_empleados_limpio['sector']).sum()

,anio,mes,sector,empleados
sector,,,,
Confeccionista,0,0,0,0
Electrónicas,0,0,0,0
Otras,0,0,0,0
Pesqueras,0,0,0,0
Plásticas,0,0,0,0
Textiles,0,0,0,0


In [33]:
# Verifico valores únicos
print(sorted(df_empleados_limpio['mes'].unique()))

['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']


In [34]:
# Muestro los primeros registros de cada sector para verificar desde qué año arranca cada uno
for sector in df_empleados_limpio['sector'].unique():
    print(f"\nPrimeras filas del sector: {sector}")
    display(df_empleados_limpio[df_empleados_limpio['sector'] == sector].head())


Primeras filas del sector: Confeccionista


,anio,mes,sector,empleados
0,2001,enero,Confeccionista,326
1,2001,febrero,Confeccionista,323
2,2001,marzo,Confeccionista,333
3,2001,abril,Confeccionista,337
4,2001,mayo,Confeccionista,346



Primeras filas del sector: Electrónicas


,anio,mes,sector,empleados
291,2001,enero,Electrónicas,2652
292,2001,febrero,Electrónicas,2624
293,2001,marzo,Electrónicas,2629
294,2001,abril,Electrónicas,2249
295,2001,mayo,Electrónicas,2323



Primeras filas del sector: Plásticas


...

,anio,mes,sector,empleados
873,2001,enero,Textiles,274
874,2001,febrero,Textiles,283
875,2001,marzo,Textiles,286
876,2001,abril,Textiles,283
877,2001,mayo,Textiles,306



Primeras filas del sector: Pesqueras


,anio,mes,sector,empleados
1164,2001,enero,Pesqueras,353
1165,2001,febrero,Pesqueras,278
1166,2001,marzo,Pesqueras,275
1167,2001,abril,Pesqueras,273
1168,2001,mayo,Pesqueras,262



Primeras filas del sector: Otras


,anio,mes,sector,empleados
1455,2001,enero,Otras,90
1456,2001,febrero,Otras,92
1457,2001,marzo,Otras,94
1458,2001,abril,Otras,103
1459,2001,mayo,Otras,107


In [35]:
# Tipo de datos y cantidad de valores no nulos
df_empleados_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   anio       1746 non-null   int32 
 1   mes        1746 non-null   object
 2   sector     1746 non-null   object
 3   empleados  1746 non-null   int32 
dtypes: int32(2), object(2)
memory usage: 41.1+ KB


## 4. Dataset de Establecimientos Industriales por Sector

Este dataset contiene la cantidad de establecimientos industriales activos por sector, año y mes.  
A partir del archivo original en formato ancho, lo transformo, normalizo y limpio para dejarlo listo para su integración con los otros datasets del proyecto.

### 4.1 Carga del archivo y selección de columnas

Cargo el archivo Excel original, selecciono solo las columnas útiles y renombro los encabezados.

In [36]:
# Cargo el archivo
archivo_estab = ruta_base + '14_3_02_Establecimientos_industriales_rama-1.xlsx'
df_estab = pd.read_excel(archivo_estab, skiprows=3)

# Corto columnas de interés
df_estab = df_estab.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 9]]  
df_estab.columns = ['anio', 'mes', 'total', 'confeccionistas', 'electronicas',
                    'plasticas', 'textiles', 'pesqueras', 'otras']

### 4.2 Relleno de valores y eliminación de filas incompletas

Relleno los años faltantes y elimino las filas vacías o mal estructuradas.


In [37]:
# Convierto a numérico y reemplazo valores inválidos (como '- Cero absoluto')
df_estab['anio'] = pd.to_numeric(df_estab['anio'], errors='coerce')

# Relleno los años faltantes hacia abajo
df_estab['anio'] = df_estab['anio'].ffill().astype(int)

# Elimino filas que no tengan mes o año (completamente vacías)
df_estab = df_estab.dropna(subset=['anio', 'mes'], how='any')

### 4.3 Transformación a formato largo

Transformo el dataset de formato ancho a largo y capitalizo los nombres de los sectores.

In [38]:
df_estab_limpio = df_estab.melt(
    id_vars=['anio', 'mes'],
    var_name='sector',
    value_name='establecimientos'
)

# Capitalizo los nombres de los sectores
df_estab_limpio['sector'] = df_estab_limpio['sector'].str.capitalize()

# Mostrar primeras filas para confirmar estructura
print("Dataset de establecimientos transformado:")
display(df_estab_limpio.head(15))

Dataset de establecimientos transformado:


,anio,mes,sector,establecimientos
0,2001,Enero,Total,50.0
1,2001,Febrero,Total,47.0
2,2001,Marzo,Total,47.0
3,2001,Abril,Total,47.0
4,2001,Mayo,Total,47.0
5,2001,Junio,Total,47.0
6,2001,Julio,Total,47.0
7,2001,Agosto,Total,47.0
8,2001,Septiembre,Total,47.0
9,2001,Octubre,Total,47.0


### 4.4 Diagnóstico del dataset en formato largo

Una vez transformado el dataset a formato largo y capitalizados los nombres de sector, realizo un diagnóstico inicial para detectar inconsistencias.  
Analizo tipos de datos, presencia de valores nulos, duplicados y la distribución de valores en las columnas `anio`, `mes`, `sector` y `establecimientos`.

In [39]:
# Estructura general del DataFrame
df_estab_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2037 entries, 0 to 2036
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   anio              2037 non-null   int32  
 1   mes               2037 non-null   object 
 2   sector            2037 non-null   object 
 3   establecimientos  2037 non-null   float64
dtypes: float64(1), int32(1), object(2)
memory usage: 55.8+ KB


In [40]:
# Cantidad de valores nulos por columna
df_estab_limpio.isnull().sum()

anio                0
mes                 0
sector              0
establecimientos    0
dtype: int64

In [41]:
# Cantidad de registros duplicados
df_estab_limpio.duplicated().sum()

0

In [42]:
# Distribución de valores en la columna 'sector'
df_estab_limpio['sector'].value_counts().sort_index()

sector
Confeccionistas    291
Electronicas       291
Otras              291
Pesqueras          291
Plasticas          291
Textiles           291
Total              291
Name: count, dtype: int64

In [43]:
# Distribución de valores en la columna 'mes'
df_estab_limpio['mes'].value_counts().sort_index()

mes
Abril            7
Abril*           7
Agosto           7
Agosto*          7
Diciembre       14
Diciembre*       7
Enero            7
Enero*          14
Febrero          7
Febrero*        14
Julio            7
Julio*           7
Junio            7
Junio*           7
Mar*             7
Marzo            7
Marzo*           7
Mayo             7
Mayo*            7
Noviembre       14
Noviembre*       7
Octubre         14
Octubre*         7
Septiembre      14
Septiembre*      7
abril          140
abril           14
agosto         140
agosto          14
diciembre      140
diciembre        7
enero          140
enero           14
febrero        140
febrero         14
julio          140
julio           14
junio          140
junio           14
marzo          140
marzo           14
mayo           140
mayo            14
noviembre      140
noviembre        7
octubre        140
octubre          7
septiembre     140
septiembre       7
Name: count, dtype: int64

In [44]:
# Distribución de valores en la columna 'establecimientos'
df_estab_limpio['establecimientos'].value_counts().sort_index()

establecimientos
0.0       3
1.0      61
2.0     126
3.0     218
4.0     182
5.0      85
6.0     188
7.0     208
8.0     288
9.0      96
16.0     27
17.0      6
18.0     15
19.0     19
20.0     46
21.0     14
22.0     22
23.0     25
24.0     28
25.0     14
26.0      6
27.0     35
28.0     34
32.0      1
37.0      3
38.0      4
39.0      4
40.0      7
42.0      3
43.0      2
44.0     14
45.0     18
46.0      4
47.0     32
48.0      8
49.0     39
50.0     21
51.0      6
52.0      5
53.0     15
54.0     18
55.0     16
56.0     36
57.0     35
Name: count, dtype: int64

### 4.5 Limpieza de la columna `mes`

Normalizo todos los valores de la columna `mes` para que sean consistentes.  
Elimino símbolos, espacios, convierto a minúsculas y unifico los nombres según el calendario.  
Esto asegura que luego puedan ordenarse cronológicamente sin errores.

In [45]:
# Diccionario con los nombres limpios
meses_validos = {
    'ene': 'enero', 'feb': 'febrero', 'mar': 'marzo',
    'abr': 'abril', 'may': 'mayo', 'jun': 'junio',
    'jul': 'julio', 'ago': 'agosto', 'sep': 'septiembre',
    'oct': 'octubre', 'nov': 'noviembre', 'dic': 'diciembre'
}

# Limpieza unificada
df_estab_limpio['mes'] = (
    df_estab_limpio['mes']
    .astype(str)
    .str.lower()
    .str.strip()
    .str.replace(r'\*', '', regex=True)
    .str.replace(r'\s+', '', regex=True)  # quita espacios intermedios
    .map(lambda x: next((v for k, v in meses_validos.items() if x.startswith(k)), x))
)

In [46]:
# Verificación
print(sorted(df_estab_limpio['mes'].unique()))

['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']


### 4.6 Exclusión del sector `Total`

El sector `'Total'` representa una suma de todos los sectores.  
Como ya analizamos individualmente cada sector, eliminamos este grupo agregado para no duplicar información.

In [47]:
# Filtro el DataFrame, excluyendo el sector 'Total'
df_estab_limpio = df_estab_limpio[df_estab_limpio['sector'].str.lower() != 'total']

In [48]:
print(sorted(df_estab_limpio['sector'].unique()))


['Confeccionistas', 'Electronicas', 'Otras', 'Pesqueras', 'Plasticas', 'Textiles']


### 4.7 Corrección de valores anómalos en el sector `Pesqueras`

En el sector `Pesqueras`, detecto valores igual a 0 en algunos años específicos.  
Asumo que se trata de errores de carga o valores faltantes, por lo que los imputo con el promedio redondeado del mes anterior y posterior.

In [49]:
# Convierto a entero por seguridad (en caso de que no lo esté)
df_estab_limpio['establecimientos'] = df_estab_limpio['establecimientos'].astype(int)

# Reinicio el índice para evitar errores en el acceso por posición
df_estab_limpio.reset_index(drop=True, inplace=True)

# Imputo los valores 0 para Pesqueras
for idx in df_estab_limpio[
    (df_estab_limpio['sector'] == 'Pesqueras') & (df_estab_limpio['establecimientos'] == 0)
].index:
    
    try:
        val_ant = df_estab_limpio.loc[idx - 1, 'establecimientos']
        val_sig = df_estab_limpio.loc[idx + 1, 'establecimientos']
        
        if np.isfinite(val_ant) and np.isfinite(val_sig):
            promedio_redondeado = round((val_ant + val_sig) / 2)
            df_estab_limpio.at[idx, 'establecimientos'] = promedio_redondeado
    except:
        pass

In [50]:
# Verifico que ya no haya 0 en diciembre de esos años
print(df_estab_limpio[
    (df_estab_limpio['sector'] == 'Pesqueras') &
    (df_estab_limpio['anio'].isin([2001, 2002, 2004])) &
    (df_estab_limpio['mes'] == 'diciembre')
])

      anio        mes     sector  establecimientos
1175  2001  diciembre  Pesqueras                 2
1187  2002  diciembre  Pesqueras                 2
1211  2004  diciembre  Pesqueras                 2


In [51]:
print("Valores unicos de establecimientos:")
# Mostrar valores únicos en la columna 'establecimientos'
print(df_estab_limpio['establecimientos'].unique())

Valores unicos de establecimientos:
[ 6  5  7  4  3 22 21 20 17 18 16 19 23 24 25 26 27 28  8  9  2  1]


### 4.8 Diagnóstico posterior a la limpieza

Finalizada la limpieza, realizo un diagnóstico general del DataFrame para verificar su integridad.  
Me aseguro de que no queden valores nulos o duplicados, reviso los tipos de datos y la distribución  
de registros por sector, mes y cantidad de establecimientos.


In [52]:
# Revisión de estructura y tipos
df_estab_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   anio              1746 non-null   int32 
 1   mes               1746 non-null   object
 2   sector            1746 non-null   object
 3   establecimientos  1746 non-null   int32 
dtypes: int32(2), object(2)
memory usage: 41.1+ KB


In [53]:
# Verificar si quedan valores nulos
df_estab_limpio.isnull().sum()

anio                0
mes                 0
sector              0
establecimientos    0
dtype: int64

In [54]:
# Verificar si hay filas duplicadas
df_estab_limpio.duplicated().sum()

0

In [55]:
# Registros por sector
df_estab_limpio['sector'].value_counts().sort_index()

sector
Confeccionistas    291
Electronicas       291
Otras              291
Pesqueras          291
Plasticas          291
Textiles           291
Name: count, dtype: int64

In [56]:
# Registros por mes
df_estab_limpio['mes'].value_counts()

mes
enero         150
febrero       150
marzo         150
abril         144
mayo          144
junio         144
julio         144
agosto        144
septiembre    144
octubre       144
noviembre     144
diciembre     144
Name: count, dtype: int64

In [57]:
# Muestro los primeros registros de cada sector para verificar desde qué año arranca cada uno
for sector in df_estab_limpio['sector'].unique():
    print(f"\nPrimeras filas del sector: {sector}")
    display(df_estab_limpio[df_estab_limpio['sector'] == sector].head())


Primeras filas del sector: Confeccionistas


,anio,mes,sector,establecimientos
0,2001,enero,Confeccionistas,6
1,2001,febrero,Confeccionistas,6
2,2001,marzo,Confeccionistas,6
3,2001,abril,Confeccionistas,6
4,2001,mayo,Confeccionistas,6



Primeras filas del sector: Electronicas


,anio,mes,sector,establecimientos
291,2001,enero,Electronicas,22
292,2001,febrero,Electronicas,21
293,2001,marzo,Electronicas,21
294,2001,abril,Electronicas,20
295,2001,mayo,Electronicas,20



Primeras filas del sector: Plasticas


...

,anio,mes,sector,establecimientos
873,2001,enero,Textiles,8
874,2001,febrero,Textiles,8
875,2001,marzo,Textiles,8
876,2001,abril,Textiles,8
877,2001,mayo,Textiles,8



Primeras filas del sector: Pesqueras


,anio,mes,sector,establecimientos
1164,2001,enero,Pesqueras,2
1165,2001,febrero,Pesqueras,1
1166,2001,marzo,Pesqueras,1
1167,2001,abril,Pesqueras,1
1168,2001,mayo,Pesqueras,1



Primeras filas del sector: Otras


,anio,mes,sector,establecimientos
1455,2001,enero,Otras,4
1456,2001,febrero,Otras,4
1457,2001,marzo,Otras,4
1458,2001,abril,Otras,4
1459,2001,mayo,Otras,4


## 5. Verificación de correcta transformación de datasets

Antes de realizar la unión de los tres datasets, realizo una verificación cruzada para asegurarme de que todos tienen la misma estructura de claves (`anio`, `mes`, `sector`)  
y que sus registros son compatibles entre sí. Esto me permite anticipar errores o pérdidas de información al hacer los `merge`.

In [58]:
# Función para revisar cualquier dataframe
def verificar_dataframe(df, nombre, valor_col):
    print(f"\nVerificando {nombre}:")
    print(f"Columnas: {df.columns.tolist()}")

    # Revisión de estructura y tipos
    print("\nEstructura:")
    print(df.info())

    # Nulos
    print("\n¿Nulos por columna?")
    print(df.isna().sum())

    # Valores únicos
    print("\nMeses únicos:", sorted(df['mes'].unique()))
    print("Sectores únicos:", sorted(df['sector'].unique()))

    # Ejemplo de filas
    print("\nPreview:")
    display(df.head(3))

# Llamadas por dataset
verificar_dataframe(df_produccion_total, "Producción", "Produccion")
verificar_dataframe(df_empleados_limpio, "Empleados", "empleados")
verificar_dataframe(df_estab_limpio, "Establecimientos", "establecimientos")



Verificando Producción:
Columnas: ['anio', 'mes', 'sector', 'Produccion']

Estructura:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   anio        881 non-null    int32 
 1   mes         881 non-null    object
 2   sector      881 non-null    object
 3   Produccion  881 non-null    int32 
dtypes: int32(2), object(2)
memory usage: 20.8+ KB
None

¿Nulos por columna?
anio          0
mes           0
sector        0
Produccion    0
dtype: int64

Meses únicos: ['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']
Sectores únicos: ['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']

Preview:


,anio,mes,sector,Produccion
0,2013,enero,Confeccionista,219953
1,2013,febrero,Confeccionista,478428
2,2013,marzo,Confeccionista,455248



Verificando Empleados:
Columnas: ['anio', 'mes', 'sector', 'empleados']

Estructura:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   anio       1746 non-null   int32 
 1   mes        1746 non-null   object
 2   sector     1746 non-null   object
 3   empleados  1746 non-null   int32 
dtypes: int32(2), object(2)
memory usage: 41.1+ KB
None

¿Nulos por columna?
anio         0
mes          0
sector       0
empleados    0
dtype: int64

Meses únicos: ['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']
Sectores únicos: ['Confeccionista', 'Electrónicas', 'Otras', 'Pesqueras', 'Plásticas', 'Textiles']

Preview:


,anio,mes,sector,empleados
0,2001,enero,Confeccionista,326
1,2001,febrero,Confeccionista,323
2,2001,marzo,Confeccionista,333



Verificando Establecimientos:
Columnas: ['anio', 'mes', 'sector', 'establecimientos']

Estructura:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   anio              1746 non-null   int32 
 1   mes               1746 non-null   object
 2   sector            1746 non-null   object
 3   establecimientos  1746 non-null   int32 
dtypes: int32(2), object(2)
memory usage: 41.1+ KB
None

¿Nulos por columna?
anio                0
mes                 0
sector              0
establecimientos    0
dtype: int64

Meses únicos: ['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']
Sectores únicos: ['Confeccionistas', 'Electronicas', 'Otras', 'Pesqueras', 'Plasticas', 'Textiles']

Preview:


,anio,mes,sector,establecimientos
0,2001,enero,Confeccionistas,6
1,2001,febrero,Confeccionistas,6
2,2001,marzo,Confeccionistas,6


In [59]:
print(sorted(df_empleados_limpio['sector'].unique()))
print(sorted(df_estab_limpio['sector'].unique()))
print(sorted(df_produccion_total['sector'].unique()))

['Confeccionista', 'Electrónicas', 'Otras', 'Pesqueras', 'Plásticas', 'Textiles']
['Confeccionistas', 'Electronicas', 'Otras', 'Pesqueras', 'Plasticas', 'Textiles']
['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']


In [60]:
# Reemplazo y unifico los nombres de los sectores en los tres dataframes
mapa_sectores = {
    'Confeccionista': 'Confeccionista',
    'Confeccionistas': 'Confeccionista',
    'Electrónica': 'Electronica',
    'Electrónicas': 'Electronica',
    'Electronicas': 'Electronica',
    'Pesquera': 'Pesquera',
    'Pesqueras': 'Pesquera',
    'Plástica': 'Plastica',
    'Plásticas': 'Plastica',
    'Plasticas': 'Plastica',
    'Textil': 'Textil',
    'Textiles': 'Textil',
    'Otro': 'Otros',
    'Otros': 'Otros',
    'Otras': 'Otros'
}

for df in [df_produccion_total, df_empleados_limpio, df_estab_limpio]:
    df['sector'] = df['sector'].replace(mapa_sectores)



In [61]:
print(sorted(df_empleados_limpio['sector'].unique()))
print(sorted(df_estab_limpio['sector'].unique()))
print(sorted(df_produccion_total['sector'].unique()))

['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']
['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']
['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']


## 6. Unificación de los tres datasets

Luego de normalizar y verificar la integridad de los tres datasets (`producción`, `empleados` y `establecimientos`), realizo la unión final utilizando las columnas clave: `anio`, `mes` y `sector`.

En lugar de restringir la unión a los registros que aparecen en los tres datasets (`how='inner'`), utilizo `how='outer'` para conservar el historial completo, incluyendo los registros que aún no tienen valores de producción.  

Esta decisión me permite posteriormente separar los registros en dos conjuntos:
- Uno con información completa para el entrenamiento del modelo.
- Otro con valores faltantes en producción, que se utilizarán como casos de testeo o predicción futura.


In [62]:
# Unimos los datasets por anio, mes y sector
df_unificado = df_produccion_total.merge(
    df_empleados_limpio, on=['anio', 'mes', 'sector'], how='outer'
).merge(
    df_estab_limpio, on=['anio', 'mes', 'sector'], how='outer'
)

In [64]:
print("Dataset unificado:")
display(df_unificado.head(10))
print("\nDimensiones:", df_unificado.shape)
print("\nNulos por columna:\n", df_unificado.isna().sum())


Dataset unificado:


,anio,mes,sector,Produccion,empleados,establecimientos
0,2001,abril,Confeccionista,NaN,337,6
1,2001,abril,Electronica,NaN,2249,20
2,2001,abril,Otros,NaN,103,4
3,2001,abril,Pesquera,NaN,273,1
4,2001,abril,Plastica,NaN,289,8
5,2001,abril,Textil,NaN,283,8
6,2001,agosto,Confeccionista,NaN,351,6
7,2001,agosto,Electronica,NaN,2637,20
8,2001,agosto,Otros,NaN,113,4
9,2001,agosto,Pesquera,NaN,166,1



Dimensiones: (1746, 6)

Nulos por columna:
 anio                  0
mes                   0
sector                0
Produccion          865
empleados             0
establecimientos      0
dtype: int64


### 6.1 Ordenamiento cronológico del dataset unificado

Después de realizar el merge de los tres datasets, la columna `mes` queda en orden alfabético, ya que pandas la interpreta como texto (`object`) y no reconoce el orden natural del calendario.  

Para corregir esto, convierto `mes` a una categoría ordenada cronológicamente, ordeno el DataFrame por `anio`, `mes` y `sector`, y finalmente devuelvo `mes` a texto plano si es necesario.

In [65]:
# Definimos el orden cronológico real de los meses
orden_meses = [
    'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
    'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre'
]

# Convertimos 'mes' a categoría ordenada
df_unificado['mes'] = pd.Categorical(df_unificado['mes'], categories=orden_meses, ordered=True)

# Ordenamos cronológicamente
df_unificado = df_unificado.sort_values(by=['anio', 'mes', 'sector']).reset_index(drop=True)

# Devolvemos 'mes' a tipo string
df_unificado['mes'] = df_unificado['mes'].astype(str)

In [66]:
print("Dataset unificado:")
display(df_unificado.head(10))

Dataset unificado:


,anio,mes,sector,Produccion,empleados,establecimientos
0,2001,enero,Confeccionista,NaN,326,6
1,2001,enero,Electronica,NaN,2652,22
2,2001,enero,Otros,NaN,90,4
3,2001,enero,Pesquera,NaN,353,2
4,2001,enero,Plastica,NaN,284,8
5,2001,enero,Textil,NaN,274,8
6,2001,febrero,Confeccionista,NaN,323,6
7,2001,febrero,Electronica,NaN,2624,21
8,2001,febrero,Otros,NaN,92,4
9,2001,febrero,Pesquera,NaN,278,1


### 6.2 Separación de registros para predicción (`df_test`)

Del dataset unificado, separo aquellos registros que no tienen valores en la columna `Produccion`.  
Estos no serán utilizados para entrenar el modelo, ya que no disponen de la variable objetivo, pero son muy útiles como casos reales a predecir una vez que el modelo esté entrenado.  
Se almacenan en el DataFrame `df_test`.


In [67]:
# Registros sin valores en Produccion → casos reservados para predicción
df_test = df_unificado[df_unificado['Produccion'].isna()].copy()

# Verifico cantidad de registros y muestro algunos
print("Registros reservados para predicción (df_test):", df_test.shape[0])
display(df_test.head())

Registros reservados para predicción (df_test): 865


,anio,mes,sector,Produccion,empleados,establecimientos
0,2001,enero,Confeccionista,NaN,326,6
1,2001,enero,Electronica,NaN,2652,22
2,2001,enero,Otros,NaN,90,4
3,2001,enero,Pesquera,NaN,353,2
4,2001,enero,Plastica,NaN,284,8


In [68]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   anio              1746 non-null   int32  
 1   mes               1746 non-null   object 
 2   sector            1746 non-null   object 
 3   Produccion        881 non-null    float64
 4   empleados         1746 non-null   int32  
 5   establecimientos  1746 non-null   int32  
dtypes: float64(1), int32(3), object(2)
memory usage: 61.5+ KB


### 6.3 Creación del dataset final para modelado (`df_final`)

Mantengo `df_unificado` como respaldo completo del merge, incluyendo registros sin producción.  
A partir de este, creo un nuevo DataFrame `df_final`, que contiene únicamente los registros con valores válidos de producción.  
Este dataset será el utilizado para entrenar y evaluar los modelos de aprendizaje supervisado.

In [71]:
# Creo df_final con solo los registros completos
df_final = df_unificado[df_unificado['Produccion'].notna()].copy()

# Convierto Produccion nuevamente a entero
df_final['Produccion'] = df_final['Produccion'].astype(int)

# Verifico estructura final
print("Dataset final para modelado (df_final):")
display(df_final.head())
print("\nDimensiones:", df_final.shape)
print("\nNulos por columna:\n", df_final.isna().sum())

Dataset final para modelado (df_final):


,anio,mes,sector,Produccion,empleados,establecimientos
864,2013,enero,Confeccionista,219953,452,6
865,2013,enero,Electronica,1260521,11001,27
866,2013,enero,Otros,2347676,515,3
867,2013,enero,Pesquera,2056519,415,3
868,2013,enero,Plastica,7415631,795,7



Dimensiones: (881, 6)

Nulos por columna:
 anio                0
mes                 0
sector              0
Produccion          0
empleados           0
establecimientos    0
dtype: int64


In [72]:
# Reinicio el índice de df_final para que comience desde 0
df_final.reset_index(drop=True, inplace=True)

In [73]:
df_final

,anio,mes,sector,Produccion,empleados,establecimientos
0,2013,enero,Confeccionista,219953,452,6
1,2013,enero,Electronica,1260521,11001,27
2,2013,enero,Otros,2347676,515,3
3,2013,enero,Pesquera,2056519,415,3
4,2013,enero,Plastica,7415631,795,7
...,...,...,...,...,...,...
876,2025,marzo,Confeccionista,216133,238,4
877,2025,marzo,Electronica,887363,6731,19
878,2025,marzo,Otros,8844,435,1
879,2025,marzo,Pesquera,513161,219,2


### 6.4 Exportación de datasets procesados a CSV

Una vez finalizada la unificación, limpieza y separación de los datos, procedo a exportar los distintos DataFrames en archivos CSV.

- Los archivos con sufijo `_v2` corresponden a versiones actualizadas de datasets ya procesados.
- `dataset_final.csv` contiene los registros listos para el modelado supervisado.
- `dataset_test_real.csv` contiene los registros sin datos de producción, destinados a pruebas de predicción.

Todos los archivos se guardan en la carpeta `../data/processed/`.


In [75]:
# Exportación de datasets procesados
df_produccion_total.to_csv('../data/processed/produccion_total_por_sector_v2.csv', index=False)
df_empleados_limpio.to_csv('../data/processed/empleados_total_por_sector_v2.csv', index=False)
df_estab_limpio.to_csv('../data/processed/establecimientos_total_por_sector_v2.csv', index=False)
df_unificado.to_csv('../data/processed/dataset_unificado_industria_v2.csv', index=False)

# Nuevos datasets
df_test.to_csv('../data/processed/dataset_test_real.csv', index=False)
df_final.to_csv('../data/processed/dataset_final.csv', index=False)

Verifico la correcta exportacion

In [76]:
import os

print(os.listdir('../data/processed/'))


['.gitkeep', 'dataset_final.csv', 'dataset_test_real.csv', 'dataset_unificado_industria.csv', 'dataset_unificado_industria_v2.csv', 'empleados_total_por_sector.csv', 'empleados_total_por_sector_v2.csv', 'establecimientos_total_por_sector.csv', 'establecimientos_total_por_sector_v2.csv', 'produccion_total_por_sector.csv', 'produccion_total_por_sector_v2.csv']
